YOU CAN EXECUTE THE COMMAND pip install -r TO INSTALL THE REQUIREMENTS IN THE FOLLOW PROJECT

In [ ]:
%pip install -r requirements.txt

OR INSTALL ANY REQUIREMENT THAT YOU NEED TO USE IN THE PROJECT

In [ ]:
%pip install beautifulsoup4
%pip install requests
%pip install pandas

In [3]:
import bs4 as bs
import requests
import pandas as pd
import json


# Scrapes using Beautiful Soup and Requests


In [9]:
response = requests.get("https://fakestoreapi.com/products")

response.raise_for_status()

response = response.text

soup = bs.BeautifulSoup(response, 'html')



TRANSFORMS THE DATA (STRING) TO A JSON

In [10]:
ecommerce_data = json.loads(str(soup.text))


# Creates an dataframe for the products

In [ ]:
products_df = pd.DataFrame(ecommerce_data)
display(products_df)

# Creates an dataframe for the ratings

In [ ]:
ratings_df = []
count = 1
for i in ecommerce_data:
  rating_json = {
      'id': count,
      'rate': i['rating']['rate'],
      'count': i['rating']['count']
  }
  count = count + 1
  ratings_df.append(rating_json)

ratings_df = pd.DataFrame(ratings_df)
display(ratings_df)



# Creates an dataframe for the categories

In [ ]:
categories_df = []
names =[]
count = 1
for i in ecommerce_data:
  if i['category'] not in names:
    category_json = {
        'id': count,
        'name': i['category']
    }
    names.append(i['category'])
    categories_df.append(category_json)
    count = count + 1

categories_df = pd.DataFrame(categories_df)

display(categories_df)


# Normalize the products dataframe

Creates the field products.category_id

In [14]:
for i, product_row in products_df.iterrows():
  for j, category_row in categories_df.iterrows():
    if product_row['category'] == category_row['name']:
      products_df.at[i, 'category_id'] = category_row['id']
      break


Removes the field products.category



In [ ]:
products_df.drop(columns=['category'], inplace=True)
display(products_df)

Creates the field products.rating_id

In [16]:

for i, product_row in products_df.iterrows():
  for j, rating_row in ratings_df.iterrows():
    if product_row['rating']['rate'] == rating_row['rate'] and product_row['rating']['count'] == rating_row['count']:
      products_df.at[i, 'rating_id'] = rating_row['id']
      break


In [ ]:
display(products_df)

Removes the field products.rating


In [18]:
products_df.drop(columns=['rating'], inplace=True)


In [ ]:
display(products_df)

# EXPORT THE DATAFRAMES TO CSV AND XLSX FORMATS

In [63]:
import os



### CSV

In [64]:
dfs = {
    "products": products_df,
    "categories": categories_df,
    "ratings": ratings_df
}

os.makedirs("csv_files", exist_ok=True)

for name, dataframe in dfs.items():
    dataframe.to_csv(f"csv_files/{name}.csv", index=False)


### XLSX 

In [ ]:
%pip install openpyxl

In [67]:

dfs = {
    "products": products_df,
    "categories": categories_df,
    "ratings": ratings_df
}

os.makedirs("xlsx_files", exist_ok=True)

with pd.ExcelWriter("xlsx_files/store.xlsx", engine="openpyxl") as writer:
    for name, dataframe in dfs.items():
        dataframe.to_excel(writer, sheet_name=name, index=False)

# CONNECTION TO THE MYSQL ACCOUNT

In [ ]:
%pip install mysql-connector-python

import getpass
import mysql.connector

## INSERT YOUR USERDATA FROM MYSQL

In [ ]:
user = input("insert your username")
password = getpass.getpass("insert your password")
host = input("insert your host. (e.g. localhost)")
port = int(input("insert your port. (e.g. 3306)"))

#METHOD TO VALIDATE HOST
def validate_host(host):
    parts = host.split('.')
    if len(parts) != 4:
      raise ValueError
    for part in parts:
      if not part.isdigit():
        return False
      num = int(part)
      if num < 0 or num > 255:
        return False
    return True

#DATA VALIDATION
if user is None:
  raise ValueError("The user must be inserted")

if password is None:
  raise ValueError("The password must be inserted")

if host != "localhost" and not validate_host(host):
  raise ValueError("The host must be IP or localhost")

if port < 1 or port > 65535:
  raise ValueError("The port must be a number")

## CREATE A CONNECTION TO THE MYSQL ACCOUNT

In [50]:
conn = mysql.connector.connect(
        host=host,
        user=user,
        password=password
    )
cursor = conn.cursor()

## CLOSE CONNECTION TO THE MYSQL ACCOUNT 

In [52]:
cursor.close()
conn.close()

# CREATES A DATABASE IN MYSQL



In [ ]:
try:
   
            #CREATES THE DATABASE 
    cursor.execute(f"CREATE DATABASE IF NOT EXISTS store;")
    
    print(f"STORE DATABASE SUCCESSFULLY CREATED.")
    
    cursor.execute("USE store")

            #CREATES THE TABLES
    cursor.execute(
        """
            CREATE TABLE IF NOT EXISTS categories (
                id INT PRIMARY KEY AUTO_INCREMENT,
                name VARCHAR(255) NOT NULL
            );
        """
    )
    
    cursor.execute(
        """
            CREATE TABLE IF NOT EXISTS ratings (
                id INT PRIMARY KEY AUTO_INCREMENT,
                rate FLOAT,
                count INT
            ); 
        """
    )

    cursor.execute(
        """
            CREATE TABLE IF NOT EXISTS products (
                id INT PRIMARY KEY AUTO_INCREMENT,
                name VARCHAR(255) NOT NULL,
                price FLOAT,
                description VARCHAR(1000) NOT NULL,
                image VARCHAR(200) NOT NULL,
                category_id INT,
                rating_id INT,
                FOREIGN KEY (category_id) REFERENCES categories(id),
                FOREIGN KEY (rating_id) REFERENCES ratings(id)
            );
        """
    )       
     

    for _, row in categories_df.iterrows():
        cursor.execute(
            """
                INSERT IGNORE INTO categories (id, name)
                VALUES (%s, %s)
            """, (row['id'], row['name'])
        )
    
    for _, row in ratings_df.iterrows():
        cursor.execute(
            """
                INSERT IGNORE INTO ratings (id, rate,count)
                VALUES (%s, %s, %s)
            """, (row['id'], row['rate'],row['count'])
        )
    
    for _, row in products_df.iterrows():
        cursor.execute(
            """
                INSERT IGNORE INTO products (id, name,price,description,image,category_id,rating_id)                
                VALUES (%s, %s, %s, %s,%s,%s,%s)
            """, (row['id'], row['title'],row['price'],row['description'],row['image'],row['category_id'],row['rating_id'])
        )
    conn.commit()
    
except mysql.connector.Error as err:
    print(" ERROR:", err.msg)
    conn.rollback()



# DATA VISUALIZATION 

In [ ]:
%pip install matplotlib 

import matplotlib.pyplot as plt


### HIGHEST RATED PRODUCTS

In [ ]:
query = """
    SELECT p1.name, p2.rate
    FROM products AS p1
    INNER JOIN ratings AS p2
        ON p1.rating_id = p2.id
    ORDER BY rate DESC, count DESC
    LIMIT 5;
"""

cursor.execute(query)


rows = cursor.fetchall()
df = pd.DataFrame(rows, columns=["name", "rate"])

df = df.sort_values("rate", ascending=True)  
plt.figure(figsize=(10, 6))
plt.barh(df["name"], df["rate"])
plt.xlabel("Rate")
plt.title("HIGHEST RATED PRODUCTS")
plt.tight_layout()

plt.show()

### LOWEST RATED PRODUCTS

In [ ]:
query = """
        SELECT p1.name,p2.rate FROM products as p1
        INNER JOIN ratings as p2
        ON p1.rating_id = p2.id
        ORDER BY rate ASC, count DESC
        LIMIT 5
"""

cursor.execute(query)

rows = cursor.fetchall()
df = pd.DataFrame(rows, columns=["name", "rate"])

df = df.sort_values("rate", ascending=False)  

plt.figure(figsize=(10, 6))
plt.barh(df["name"], df["rate"])
plt.xlabel("Rate")
plt.title("LOWEST RATED PRODUCTS")
plt.tight_layout()

plt.show()

# VISUALIZATION BY CATEGORIES

SHOW CATEGORIES AND IDS

In [37]:
query = """
    SELECT id, name from categories
"""

cursor.execute(query)

rows = cursor.fetchall()
df = pd.DataFrame(rows, columns=["id", "name"])

display(df)


,id,name
0,1,men's clothing
1,2,jewelery
2,3,electronics
3,4,women's clothing


#### SHOW HIGHEST RATED PRODUCTS BY CATEGORY

In [38]:
categories_list = df['id'].to_list()
category = int(input("Insert the category ID"))

if category not in categories_list:
    raise ValueError('Invalid category ID')

In [ ]:
query = '''
    SELECT p1.name, 
        p2.rate,
        p3.name as category 
    FROM products AS p1 
    INNER JOIN ratings AS p2 
        ON p1.rating_id = p2.id 
    INNER JOIN categories AS p3
        ON p1.category_id = p3.id
    WHERE p1.category_id = (%s) 
    ORDER BY rate DESC, count DESC 
    LIMIT 5;
'''

params = [category]

cursor.execute(query,params)

rows = cursor.fetchall()

df = pd.DataFrame(rows, columns=["name","rate","category"])

#GETS THE NAME FROM THE CATEGORY
category_name = df['category'][0]

df = df.sort_values(by='rate',ascending=True)

plt.figure(figsize=(10, 6))
plt.barh(df["name"], df["rate"])
plt.xlabel("Rate")
plt.title(f"HIGHEST RATED PRODUCTS FROM CATEGORY {category_name.upper()}")
plt.tight_layout()

plt.show()



#### SHOW LOWEST RATED PRODUCTS BY CATEGORY

In [43]:
categories_list = df['id'].to_list()
category = int(input("Insert the category ID"))
if category not in categories_list:
    raise ValueError('Invalid category ID')

In [ ]:
query = '''
    SELECT p1.name, 
        p2.rate,
        p3.name as category 
    FROM products AS p1 
    INNER JOIN ratings AS p2 
    ON p1.rating_id = p2.id 
    INNER JOIN categories AS p3
    on p1.category_id = p3.id
    WHERE p1.category_id = (%s) 
    ORDER BY rate ASC, count DESC 
    LIMIT 5;
'''

params = [category]

cursor.execute(query,params)

rows = cursor.fetchall()

df = pd.DataFrame(rows, columns=["name","rate","category"])

#GETS THE NAME FROM THE CATEGORY
category_name = df['category'][0]

df = df.sort_values(by='rate',ascending=False)

plt.figure(figsize=(10, 6))
plt.barh(df["name"], df["rate"])
plt.xlabel("Rate")
plt.title(f"LOWEST RATED PRODUCTS FROM CATEGORY {category_name.upper()}")
plt.tight_layout()

plt.show()